In [44]:

import pandas as pd
import numpy as np
from src.cnn_model_improvedv2 import TimeSeriesCNNImprovedV2
from src.cnn_trainer_improved import CNNTrainerImproved
from src.utils import create_directories, plot_training_history, plot_predictions
from src.features import create_advanced_features
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import matplotlib.pyplot as plt
create_directories()


Struktura katalogów utworzona pomyślnie.


# === 2. Dane wejściowe ===

In [45]:
df = pd.read_csv("data/processed/cleaned_data.csv")
df['Date'] = pd.to_datetime(df['Date'], utc=True)
df = create_advanced_features(df)

features = [col for col in df.columns if col not in ['Date', 'Close/Last']]
target_col = 'Close/Last'

# === 3. Skalowanie i sekwencje ===

In [46]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_x.fit_transform(df[features])
y_scaled = scaler_y.fit_transform(df[[target_col]])

seq_length = 30
def create_sequence_data(data, seq_length):
    X = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
    return np.array(X), data[seq_length:]
X_seq, _ = create_sequence_data(X_scaled, seq_length)
y_seq = y_scaled[seq_length:]

X_temp, X_test, y_temp, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)
val_ratio = 0.1 / (1 - 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_ratio, shuffle=False)

# === 4. DataLoadery ===

In [47]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


batch_size = 64
train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=batch_size)
test_loader = DataLoader(TimeSeriesDataset(X_test, y_test), batch_size=batch_size)

# === 5. Model i trening ===

In [48]:
model = TimeSeriesCNNImprovedV2(input_channels=X_train.shape[2], seq_length=X_train.shape[1])
trainer = CNNTrainerImproved(model)
history = trainer.fit(train_loader, val_loader, epochs=150, early_stopping_patience=20)

Epoka 1: Train=0.01871, Val=0.02925
Epoka 2: Train=0.00382, Val=0.00323
Epoka 3: Train=0.00345, Val=0.00595
Epoka 4: Train=0.00307, Val=0.00068
Epoka 5: Train=0.00288, Val=0.00663
Epoka 6: Train=0.00343, Val=0.00128
Epoka 7: Train=0.00295, Val=0.00109
Epoka 8: Train=0.00262, Val=0.00107
Epoka 9: Train=0.00231, Val=0.00100
Epoka 10: Train=0.00254, Val=0.00098
Epoka 11: Train=0.00292, Val=0.00107
Epoka 12: Train=0.00229, Val=0.00154
Epoka 13: Train=0.00196, Val=0.00031
Epoka 14: Train=0.00212, Val=0.00041
Epoka 15: Train=0.00210, Val=0.00100
Epoka 16: Train=0.00192, Val=0.00194
Epoka 17: Train=0.00181, Val=0.00039
Epoka 18: Train=0.00192, Val=0.00040
Epoka 19: Train=0.00233, Val=0.00057
Epoka 20: Train=0.00239, Val=0.00088
Epoka 21: Train=0.00269, Val=0.00393
Epoka 22: Train=0.00227, Val=0.00025
Epoka 23: Train=0.00210, Val=0.00516
Epoka 24: Train=0.00183, Val=0.00087
Epoka 25: Train=0.00159, Val=0.00167
Epoka 26: Train=0.00174, Val=0.00085
Epoka 27: Train=0.00193, Val=0.00267
Epoka 28: 

# === 6. Ewaluacja ===

In [49]:
metrics = trainer.evaluate(X_test, y_test, scaler=scaler_y)


=== Metryki wydajności modelu ===
MSE: 731.005216
RMSE: 27.037108
MAE: 23.827563
R²: 0.810594


# === 7. Wizualizacja ===

In [ ]:
fig1 = plot_training_history(history)
fig2 = plot_predictions(y_test, trainer.predict(X_test), scaler=scaler_y)
plt.show(fig1)
plt.show(fig2)


In [1]:
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def directional_accuracy(y_true, y_pred):
    return np.mean((np.diff(y_true.flatten()) > 0) == (np.diff(y_pred.flatten()) > 0))

# Oblicz metryki
train_mae = mean_absolute_error(y_train_real, train_predictions_denorm)
test_mae = mean_absolute_error(y_test_real, test_predictions_denorm)
train_r2 = r2_score(y_train_real, train_predictions_denorm)
test_r2 = r2_score(y_test_real, test_predictions_denorm)
train_dir = directional_accuracy(y_train_real, train_predictions_denorm)
test_dir = directional_accuracy(y_test_real, test_predictions_denorm)

# Zapis danych
torch.save({
    'train_predictions': train_predictions_denorm,
    'test_predictions': test_predictions_denorm,
    'y_train_real': y_train_real,
    'y_test_real': y_test_real,
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_dates': df.index[seq_length:train_size],
    'test_dates': df.index[train_size+seq_length:],
    'metrics': {
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_dir_acc': train_dir,
        'test_dir_acc': test_dir
    }
}, 'notebooks/models/cnn/cnn_analysis_data.pth')

print("✅ Zapisano dane do cnn_analysis_data.pth")


NameError: name 'y_train_real' is not defined